In [0]:
# Initial Config
## Socket IO
Hashtag = 'muds'
## YouTube
API_KEY = '***************' # YouTube V3 API Key
channelID = 'UC8EYVG5oxBPe0zDdwpny2Rw' # YouTube Channel ID

In [0]:
# Install Socket IO for Python (Required for Google Colab - Delete comment-out if you are working on Google Colab)
#!pip install "python-socketio[client]"

In [0]:
# Import Library
import socketio
import json
import requests
import pandas as pd
import time

In [0]:
# Setup Socket IO
sio = socketio.Client()
@sio.event
def message(data):
    print('I received a message!')
@sio.on('my message')
def on_message(data):
    print('I received a message!')
@sio.event
def connect():
    print("I'm connected!")
@sio.event
def connect_error():
    print("The connection failed!")
@sio.event
def disconnect():
    print("I'm disconnected!")

In [0]:
# Connect to Comment Screen
sio.connect('https://commentscreen.com')
sio.emit('join', { 'room': Hashtag }) # Join to room (hashtag)

In [0]:
# Comment to Comment Screen Room (function)
def cs_comment(textmsg):
  jarray = { 'position': 'opt_ue', 'size': 'opt_small', 'color': '#ff0000', 'text': textmsg, 'uuid': 'ede93ada-3eb6-4a7d-b181-23c5f6d75f95', 'date': '1591418200999' }
  jsonStr = json.dumps(jarray, separators=(',', ':'))
  sio.emit('message', jsonStr)

In [0]:
# Comment to Comment Screen Room
#cs_comment("本日は晴天なり。")

In [0]:
# Setup and Get Comment ID for YouTube LiveStreaming
params = {
        'part': 'id',
        'key': API_KEY,
        'channelId': channelID,
        'eventType': 'live',
        'type': 'video',
        'order': 'viewCount',
        'fields': 'items(id(videoId))'
        }

url = 'https://www.googleapis.com/youtube/v3/search'
r = requests.get(url, headers=None, params=params).json()

vID = r.get('items')[0].get('id').get('videoId')

params = {
        'part': 'liveStreamingDetails,statistics,snippet',
        'key': API_KEY,
        'id': vID,
        'fields': 'items(id,liveStreamingDetails(activeLiveChatId,concurrentViewers,actualStartTime),' + \
                  'snippet(channelId,channelTitle,description,liveBroadcastContent,publishedAt,thumbnails,title),statistics)'
        }

url = 'https://www.googleapis.com/youtube/v3/videos'
r = requests.get(url, headers=None, params=params).json()

streamData = dict(r.get('items')[0])

chatID = streamData['liveStreamingDetails']['activeLiveChatId']

params = {
        'part': 'snippet',
        'key': API_KEY,
        'liveChatId': chatID,
        'maxResults': 200
        }

url = 'https://www.googleapis.com/youtube/v3/liveChat/messages'

In [0]:
# Setup Pandas Dataframe for Processing Comments
df = pd.DataFrame(columns =['id', 'timestamp', 'user', 'message'])
dfrownum = 0

In [0]:
# Get Comments and Send to CommentScreen
while True:
  messages = requests.get(url, headers=None, params=params).json()
  for i in range(len(messages['items'])):
    df_id = str(messages['items'][i]['id'])
    df_timestamp = str(messages['items'][i]['snippet']['publishedAt'])
    df_user = str(messages['items'][i]['snippet']['authorChannelId'])
    df_message = str(messages['items'][i]['snippet']['displayMessage'])
    for ii in range(2):
      df = df.append({'id': df_id, 'timestamp': df_timestamp, 'user': df_user, 'message': df_message}, ignore_index=True)
  dfcol = df.columns
  dfcol_filter = ['id', 'timestamp', 'user', 'message']
  for ii in range(len(dfcol)):
      if dfcol[ii] not in dfcol_filter:
            del df[dfcol[ii]]
  df = df.drop_duplicates()
  df = df.reset_index()
  dfcol = df.columns
  dfcol_filter = ['id', 'timestamp', 'user', 'message']
  for ii in range(len(dfcol)):
      if dfcol[ii] not in dfcol_filter:
            del df[dfcol[ii]]
  if (len(df) - dfrownum) != 0:
    for i in range(len(df) - dfrownum)[::-1]:
      cs_comment(df['message'][len(df) - 1 - i])
    dfrownum = len(df)
  time.sleep(2.5) # Sleep Timer (seconds, change if you need to)

In [0]:
# Save History to CSV
df.to_csv('ChatHistory.csv')